Original:https://github.com/kuangliu/pytorch-retinanet/blob/master/encoder.py

In [0]:
import torch
import math

class Anchor:
    def __init__(self, anchor_areas = [256*256.], 
                aspect_ratios = [1/1.,1/2., 2/1.],
                scale_ratios = [1/2., 1., 2/1.],
                anchor_base_sizes = [ 16. ] ):
        self.anchor_areas = anchor_areas
        self.aspect_ratios = aspect_ratios
        self.scale_ratios = scale_ratios
        self.anchor_base_sizes = anchor_base_sizes
        self.anchor_wh = self._get_anchor_wh()
        self.fm_sizes = None
        self.anchors = None
        self.num_anchors = len(self.aspect_ratios) * len(self.scale_ratios)

    def _get_anchor_wh(self):
        '''Compute anchor width and height for each feature map.
        Returns:
          anchor_wh: (tensor) anchor wh, sized [#fm, #anchors_per_cell, 2].
        '''
        anchor_wh = []
        for s in self.anchor_areas:
            for ar in self.aspect_ratios:  # w/h = ar
                h = math.sqrt(s/ar)
                w = ar * h
                for sr in self.scale_ratios:  # scale
                    anchor_h = h*sr
                    anchor_w = w*sr
                    anchor_wh.append([anchor_w, anchor_h])
        return torch.Tensor(anchor_wh).view(len(self.anchor_areas), -1, 2)

    def get_anchor_boxes(self, fm_sizes):
        '''Compute anchor boxes for each feature map.
        Args:
          fm_size: (tensor) model feature map size.
        Returns:
          boxes: (list) anchor boxes for each feature map. Each of size [#anchors,4],
                        where #anchors = fmw * fmh * #anchors_per_cell
        '''
        
        if isinstance(fm_sizes,list):
          fm_wh = []
          for fm_size in fm_sizes:
            fm_wh.append([fm_size[3],fm_size[2]]) # B,C,H,W
          fm_sizes = torch.tensor(fm_wh)
                   
        if (self.fm_sizes is None) or (not torch.equal(fm_sizes,self.fm_sizes)):
          self.fm_sizes = fm_sizes

          boxes = []
          
          for i, fm_size in enumerate(self.fm_sizes):
              anchor_base_size = self.anchor_base_sizes[i]
              anchor_area = self.anchor_areas[i]
              fm_w, fm_h = int(fm_size[0]), int(fm_size[1]) 
              h_grid, w_grid = torch.meshgrid(torch.arange(fm_h),torch.arange(fm_w))
              xy = torch.cat([w_grid.unsqueeze(2),h_grid.unsqueeze(2)],2).float()
              xy = xy
              xy = (xy * (anchor_base_size) + (anchor_base_size - 1.)*0.5).view(fm_h,fm_w,1,2).expand(fm_h,fm_w, self.num_anchors,2)
              wh = self.anchor_wh[i].view(1,1,self.num_anchors,2).expand(fm_h,fm_w,self.num_anchors,2)
              box = torch.cat([xy - (wh - 1.) * 0.5, xy + (wh - 1.) * 0.5], 3)  # [x1,y1,x2,y2]
              boxes.append(box.view(-1,4))
          boxes = torch.cat(boxes, 0)
          self.anchors = boxes
        return self.anchors
   
def test():
  anchor = Anchor()
  fm = torch.randn(1,512,3,10).float()
  fm_size = fm.size()
  a = anchor.get_anchor_boxes([fm_size])
  print(a[0])
  print(a[9])
  print(a[9*10])
  print(a[9*10 - 9])
  
  fm = torch.randn(1,512,3,5).float()
  fm_size = fm.size()
  b = anchor.get_anchor_boxes([fm_size])
  print(b[0])
  print(b[9])
  print(b[9*2])
  print(b[9*3])


test()

tensor([-56., -56.,  71.,  71.])
tensor([-40., -56.,  87.,  71.])
tensor([-56., -40.,  71.,  87.])
tensor([ 88., -56., 215.,  71.])
tensor([-56., -56.,  71.,  71.])
tensor([-40., -56.,  87.,  71.])
tensor([-24., -56., 103.,  71.])
tensor([ -8., -56., 119.,  71.])
